In [1]:
import pandas as pd
import numpy as np
import gensim,re,time
from gensim.models.fasttext import load_facebook_model
from nltk.tokenize import RegexpTokenizer,TreebankWordTokenizer
from code import utils
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.offline as py
import plotly.express as px
tokenizer = TreebankWordTokenizer()

pca = PCA(n_components=3)
tsne = TSNE(n_components=3)

#Paramters
TOP_N_ARTICLES = 60000
TRAIN_N = 10000
%load_ext autoreload
%autoreload 2



Number of stopwords 607


### Load data and the model 

In [2]:
df = utils.load_text_data()
pdf=utils.preprocess_text(df.head(TOP_N_ARTICLES))
tokenized_texts = [tokenizer.tokenize(txt) for txt in pdf ]

pdf[0]

'wenn du bei deinem comeback nach tagen nur drei sekunden und genau einen ballkontakt für dein erstes pflichtspieltor brauchst wahnsinns-rückkehr für marco reus bei borussia dortmund mehr als ein halbes jahr nach seinem sehnenriss im februar riesen-aufreger nach der hsv-pleite im pokal bei dynamo dresden plötzlich steigt hsv-profi toni leistner über die absperrung beimpokal-spaziergang in duisburg wird der nationalspieler in der minute eingewechselt. und trifft einen augenblick später nach einer freistoß-vorlage von jadon sancho  am tag der bvb-teenies zeigt der alte reus dass er wieder da ist. vor allemmio-neuzugang jude bellingham lässt bei seinem pflichtspiel-debüt bei trainer lucian favre die augen leuchten. nach hacken-ablage von thorgan hazard trifft der england-teenie zum . zuvor hatte landsmann sancho einen elfer zum ins tor gehämmert. noch vor der pause zaubert hazard einen freistoß direkt ins linke obere eck. die vorentscheidung. bellingham darf zur pause duschen. die teenie-

In [3]:
start_time = time.time()

fb_model = load_facebook_model("cc.de.300.bin")
fb_model.wv.most_similar(positive=['covid'])
print("--- Model loaded in %s seconds ---" % (time.time() - start_time))

--- Model loaded in 93.05387711524963 seconds ---


In [4]:
import time
start_time = time.time()
print('covid' in fb_model.wv.vocab)  # Word is out of vocabulary
print('covidioten' in fb_model.wv.vocab)  # Word is out of vocabulary
print('coronavirus' in fb_model.wv.vocab)  # Word is out of vocabulary

print("--- %s seconds ---" % (time.time() - start_time))

False
False
False
--- 0.0002789497375488281 seconds ---


In [5]:
start_time = time.time()

fb_model.build_vocab(tokenized_texts, update=True)
fb_model.train(sentences=tokenized_texts, total_examples=len(tokenized_texts), epochs=7)
print("--- %s seconds ---" % (time.time() - start_time))


--- 249.3782079219818 seconds ---


In [6]:
fb_model.wv.most_similar(positive=['corona','virus'])


[('coronavirus', 0.9802075624465942),
 ('corona-virus', 0.9771857857704163),
 ('coronavirus.', 0.9725087285041809),
 ('coronavirus’', 0.9713901281356812),
 ('coronaviren', 0.9705122113227844),
 ('coronavirusncov', 0.9698304533958435),
 ('coronaviren.', 0.9695255160331726),
 ('corona-virus.', 0.9688683748245239),
 ('coronavirus-infektion.', 0.9632284045219421),
 ('coronavirus-fall', 0.9626423120498657)]

In [7]:
sentences_from_text = [ utils.get_sentences_from_text(txt) for txt in pdf]

sentences_from_text[5]

['wenn die neue bundesliga-saison startet könnte es im dortmunder stadion endlich wieder richtig laut werden  hans-joachim watzke macht den fans hoffnung dass der auftakt gegen gladbach vor einer corona-rekord-kulisse stattfinden kann',
 'was der bvb-boss sagt und wie er die lage einschätzt das lesen sie nur mit bildplus']

In [8]:
start_time = time.time()

vectors = []
for sentences in sentences_from_text:
    ve=[]
    for sen in sentences:
        ve.append(np.array(fb_model[sen]))
    vectors.append(np.mean(ve,axis=0))
print("--- %s seconds ---" % (time.time() - start_time))
    

<ipython-input-8-3e8c68f9c425>:7: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



--- 127.41815614700317 seconds ---


In [9]:
len(vectors),vectors[0].shape

(60000, (300,))

In [11]:
p = pca.fit_transform(vectors)
p = pd.DataFrame(p,columns=["x",'y','z'])
p['headline'] = df.head(TOP_N_ARTICLES)["headline"].values


In [ ]:
from hdbscan import HDBSCAN
from sklearn.cluster import DBSCAN, MeanShift, OPTICS
from sklearn.metrics.pairwise import cosine_distances


clusterer = HDBSCAN(algorithm='best', alpha=0.8, approx_min_span_tree=False,
    gen_min_span_tree=True, leaf_size=20,
metric='euclidean', min_cluster_size=5, min_samples=2, p=None)

#clusterer = DBSCAN(metric=cosine_distances)

cluster_labels = clusterer.fit_predict(normalize(vectors,norm="l2"))
p["clusters"] = cluster_labels
fig = px.scatter(p, x="x", y="y",hover_name="headline", color = "clusters", size_max=60)
fig.update_layout(
     height=1200)

In [ ]:
clusterer.fit_predict(np.array(vectors))

In [ ]:
from gensim.models import KeyedVectors


model = KeyedVectors.load_word2vec_format("german.model", binary=False)
